<a href="https://colab.research.google.com/github/sourabh21-alt/sourabhdiwan_object_recognition/blob/master/geeksman_project1_datafetch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Introduction
Hello frnds !!Welcome to my Project-1 of Geeksman AI Squad.In this Project we will train Our ML model in such a way that when a Image of a object is passed,it is able to identify which object it is.It is able to identify 10 types of objects i.e Cat,Truck,airplane,automobile,bird,deer,dog,frog,horse and ship.
Now this Project is Divided into 3 Parts (and hence,3 notebooks).The first Notebook handles the data 
i.e download the dataset from kaggle ,modify the data as per our requirement and uploading of our dataset on Kaggle,the second one handles the download of this Dataset on our Colab and training the ML model and The Final Notebook contains a awesome Prediction function which allows you to pass your custom image and make prediction and it also contains a video of ML model predicting object.



first of all we import the necessary library

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate
import warnings
warnings.filterwarnings("ignore")

now we upload the kaggle.json file to download and upload the dataset.This kaggle.json can be generated from your Kaggle account and then the API token can be generated and saved this file in your pc/lapttop by using few easy steps.

In [3]:
from google.colab import files
my_files=files.upload()

Saving kaggle.json to kaggle.json


now,we import pandas module to read the .json file  and store the credential of this file.

In [4]:
import os
import pandas as pd
cred = pd.read_json("kaggle.json",lines=True)
os.environ['KAGGLE_USERNAME'] = cred.iloc[0][0]
os.environ['KAGGLE_KEY'] = cred.iloc[0][1]

here, we download the dataset by using api command, you have to just copy here that command.

In [5]:
! kaggle datasets download -d swaroopkml/cifar10-pngs-in-folders

 92% 129M/140M [00:04<00:00, 24.9MB/s]
100% 140M/140M [00:04<00:00, 33.2MB/s]


downloaded data file present in zip file ,so we unzip the file.

In [6]:
!unzip "*.zip"

Streaming output truncated to the last 5000 lines.
  inflating: cifar10/cifar10/train/truck/0002.png  
  inflating: cifar10/cifar10/train/truck/0003.png  
  inflating: cifar10/cifar10/train/truck/0004.png  
  inflating: cifar10/cifar10/train/truck/0005.png  
  inflating: cifar10/cifar10/train/truck/0006.png  
  inflating: cifar10/cifar10/train/truck/0007.png  
  inflating: cifar10/cifar10/train/truck/0008.png  
  inflating: cifar10/cifar10/train/truck/0009.png  
  inflating: cifar10/cifar10/train/truck/0010.png  
  inflating: cifar10/cifar10/train/truck/0011.png  
  inflating: cifar10/cifar10/train/truck/0012.png  
  inflating: cifar10/cifar10/train/truck/0013.png  
  inflating: cifar10/cifar10/train/truck/0014.png  
  inflating: cifar10/cifar10/train/truck/0015.png  
  inflating: cifar10/cifar10/train/truck/0016.png  
  inflating: cifar10/cifar10/train/truck/0017.png  
  inflating: cifar10/cifar10/train/truck/0018.png  
  inflating: cifar10/cifar10/train/truck/0019.png  
  inflating: 

here i just rename the .png file otherwise it create name collision. and i also create new folder named test_data manually in same directory of test and train data.
why rename?
read out in next comment section

In [9]:
pth=os.listdir("/content/cifar10/cifar10/test")
for i in pth:
     for count,filename in enumerate(os.listdir("/content/cifar10/cifar10/test/"+i)):
          dst=i+str(count)+".png"
          src="/content/cifar10/cifar10/test/"+i+'''/'''+filename
          dst="/content/cifar10/cifar10/test/"+i+'''/'''+dst
          os.rename(src,dst)

our testing data contains 10000 images but we dont need this much of image. so,i pick 20 images per category and upload in a single folder name test_data.
for ex:
airplane is category and it contain 1.png,2.png file name.
bird category also contain same name file.
so i pick 20 random  images from each category , there is a chance of collision of name in test_data folder,so avid the collision ,rename the file as categoryname_int_.png


In [14]:
pth=os.listdir("/content/cifar10/cifar10/test")
for i in pth:
     path=os.listdir("/content/cifar10/cifar10/test/"+i)
     for j in range(20):
          shutil.move("/content/cifar10/cifar10/test/"+i+'''/'''+path[j],"/content/cifar10/cifar10/test_data")

delete the folder name test after pick the  20 images from sub category. because rest of images not usable of us. 

In [15]:
!rm -r "/content/cifar10/cifar10/test"

now zip the file.

In [16]:
! zip -r dataset.zip  ./cifar10

Streaming output truncated to the last 5000 lines.
  adding: cifar10/cifar10/train/dog/2926.png (stored 0%)
  adding: cifar10/cifar10/train/dog/2434.png (stored 0%)
  adding: cifar10/cifar10/train/dog/4285.png (stored 0%)
  adding: cifar10/cifar10/train/dog/4853.png (stored 0%)
  adding: cifar10/cifar10/train/dog/0260.png (stored 0%)
  adding: cifar10/cifar10/train/dog/2728.png (stored 0%)
  adding: cifar10/cifar10/train/dog/4436.png (stored 0%)
  adding: cifar10/cifar10/train/dog/3015.png (stored 0%)
  adding: cifar10/cifar10/train/dog/4350.png (stored 0%)
  adding: cifar10/cifar10/train/dog/3232.png (stored 0%)
  adding: cifar10/cifar10/train/dog/1599.png (stored 0%)
  adding: cifar10/cifar10/train/dog/2960.png (stored 0%)
  adding: cifar10/cifar10/train/dog/2513.png (stored 0%)
  adding: cifar10/cifar10/train/dog/4932.png (stored 0%)
  adding: cifar10/cifar10/train/dog/3505.png (stored 0%)
  adding: cifar10/cifar10/train/dog/3871.png (stored 0%)
  adding: cifar10/cifar10/train/dog/4

mount the google drive here we put the modify dataset and upload on our kaggle id.

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


make the folder named my_data in my drive folder to 
store the zip fil

In [18]:
os.makedirs("/content/drive/My Drive/my_data")

copy the zip file in drive

In [19]:
! cp /content/dataset.zip "/content/drive/My Drive/my_data"

below 2 cell is use for upload the data in kaggle from drive. 

In [20]:
!kaggle datasets init -p "/content/drive/My Drive/my_data"

Data package template written to: /content/drive/My Drive/my_data/dataset-metadata.json


In [24]:
!kaggle datasets create -p "/content/drive/My Drive/my_data"


Starting upload for file dataset.zip
100% 118M/118M [00:02<00:00, 54.4MB/s]
Upload successful: dataset.zip (118MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/sourabhdiwan123/cifardataset


i hope u understand whatever i do hahahahahha 